In [169]:
####输入[1,3,32,280]

In [170]:
import onnx

In [175]:
onnx_model=onnx.load("../../torch_onnx/onnx_models/crnn/id_crnn_mobile_fix2_sim.onnx")

In [176]:
print(onnx.checker.check_model(onnx_model))

None


In [177]:
onnx_model.graph.input

[name: "input"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 32
      }
      dim {
        dim_value: 280
      }
    }
  }
}
, name: "cnn.conv1.rbr_reparam.weight"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 16
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 3
      }
    }
  }
}
, name: "cnn.conv1.rbr_reparam.bias"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 16
      }
    }
  }
}
, name: "cnn.blocks.0.bottleneck_conv.rbr_reparam.weight"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 16
      }
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 3
      }
    }
  }
}
, name: "cnn.blocks.0.bottleneck_conv.rbr_reparam.bias"


In [178]:
onnx_model.graph.output

[name: "output"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 70
      }
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 6086
      }
    }
  }
}
]

In [179]:
import cv2

In [180]:
src_img="../../int8_Calibration/sfzimg/rBFRM1frgsOEOBU7AAAAAIDJEhg256_name.jpg"

In [181]:
src_img=cv2.imread(src_img)

In [182]:
h,w,_=src_img.shape

In [183]:
h,w

(35, 96)

In [184]:
import sys
import torch
sys.path.append("../../../crnnmobile/")
from keys_sythtext import alphabet
from models.repvgg import repvgg_model_convert
from models.crnn_mobilev3_repvgg import CRNNSmallNoLSTM,CRNNLargeNoLSTM,CRNNSmallLSTM,CRNNLargeLSTM

In [185]:
nclass=len(alphabet)+1
print(nclass)

6086


In [186]:
import dataset as dataset

In [187]:
transformer=dataset.resizeNormalize(32,280,'val')

In [188]:
from PIL import Image

In [189]:
img = Image.fromarray(src_img).convert("RGB")

In [190]:
image=transformer(img)

In [191]:
image=image.view(1,*image.size())

In [192]:
image=image.cpu()

In [193]:
image.size()

torch.Size([1, 3, 32, 280])

In [194]:
import onnxruntime

In [195]:
ort_session=onnxruntime.InferenceSession("../../torch_onnx/onnx_models/crnn/id_crnn_mobile_fix_sim.onnx")

In [196]:
for input_meta in ort_session.get_inputs():
    print(input_meta)

NodeArg(name='input', type='tensor(float)', shape=[1, 3, 32, 280])


In [197]:
for output_meta in ort_session.get_outputs():
    print(output_meta)

NodeArg(name='output', type='tensor(float)', shape=[70, 1, 6086])


In [198]:
type(image)

torch.Tensor

In [199]:
####将tensor类型转换成numpy
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [200]:
ort_inputs={ort_session.get_inputs()[0].name:to_numpy(image)}

In [201]:
ort_inputs

{'input': array([[[[ 0.45882356,  0.45882356,  0.45882356, ..., -1.        ,
           -1.        , -1.        ],
          [ 0.45882356,  0.45882356,  0.4666667 , ..., -1.        ,
           -1.        , -1.        ],
          [ 0.45098042,  0.45882356,  0.4666667 , ..., -1.        ,
           -1.        , -1.        ],
          ...,
          [ 0.45098042,  0.4431373 ,  0.45882356, ..., -1.        ,
           -1.        , -1.        ],
          [ 0.43529415,  0.43529415,  0.45882356, ..., -1.        ,
           -1.        , -1.        ],
          [ 0.427451  ,  0.43529415,  0.45882356, ..., -1.        ,
           -1.        , -1.        ]],
 
         [[ 0.45882356,  0.45882356,  0.45098042, ..., -1.        ,
           -1.        , -1.        ],
          [ 0.45882356,  0.45882356,  0.45882356, ..., -1.        ,
           -1.        , -1.        ],
          [ 0.45098042,  0.45882356,  0.45882356, ..., -1.        ,
           -1.        , -1.        ],
          ...,
    

In [202]:
ort_outs=ort_session.run(["output"],ort_inputs)

In [203]:
pred_map=ort_outs[0]

In [204]:
pred_map

array([[[  8.375738 , -21.948593 , -18.331657 , ..., -20.201447 ,
         -12.091211 ,  -7.5625205]],

       [[ 13.010561 , -22.678034 , -16.36609  , ..., -19.192698 ,
          -8.557837 ,  -6.321658 ]],

       [[ 12.181892 , -21.081314 , -15.07355  , ..., -18.280758 ,
          -5.768859 ,  -6.0685444]],

       ...,

       [[ 12.910959 , -17.69694  , -15.144351 , ..., -17.356806 ,
         -10.10347  ,  -1.0431474]],

       [[ 11.517363 , -17.14294  , -14.264875 , ..., -16.94823  ,
          -9.5182905,  -1.8312404]],

       [[  8.806076 , -17.862976 , -14.973805 , ..., -18.38957  ,
          -9.642368 ,  -3.578435 ]]], dtype=float32)

In [205]:
pred_map.shape

(70, 1, 6086)

In [206]:
type(pred_map)

numpy.ndarray

In [207]:
pred_map=torch.Tensor(pred_map)

In [208]:
pred_map.shape

torch.Size([70, 1, 6086])

In [209]:
from torch.autograd import Variable

In [210]:
_,preds=pred_map.max(2)

In [211]:
preds.shape

torch.Size([70, 1])

In [212]:
preds=preds.squeeze(1)

In [213]:
preds.shape

torch.Size([70])

In [214]:
preds

tensor([   0,    0,    0,    0,    0,    0,  290,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 4909,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

In [215]:
confidence=[]
preds= preds.transpose(0,0).contiguous().view(-1)

In [216]:
preds.shape

torch.Size([70])

In [217]:
preds.size(0)

70

In [218]:
preds_size = Variable(torch.IntTensor([preds.size(0)]))

In [219]:
import utils

In [220]:
converter=utils.strLabelConverter(alphabet)

In [221]:
preds.data

tensor([   0,    0,    0,    0,    0,    0,  290,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 4909,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

In [222]:
sim_pred=converter.decode(preds.data,preds_size.data,raw=False)

In [223]:
sim_pred

'蒋御'